In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from coulson.data import EV_TO_KCAL, HARTREE_TO_EV, HARTREE_TO_KCAL
from coulson.interface import gen_coords_for_mol, mol_from_xyz
from rdkit import Chem
from utils import compute_mol_qualitative

plt.style.use("default")

In [ ]:
path_azaphenalene = Path(snakemake.input.path_azaphenalene) / "1.xyz"
path_pentalene = Path(snakemake.input.path_pentalene) / "IV_1.xyz"

mol_azaphenalene = mol_from_xyz(str(path_azaphenalene))
mol_pentalene = mol_from_xyz(str(path_pentalene))
mol_pentalene_ideal = mol_from_xyz(str(path_pentalene))
_ = gen_coords_for_mol(mol_pentalene_ideal)

In [ ]:
jobs = (
    (
        "azaphenalene",
        mol_azaphenalene,
        snakemake.output.azaphenalene_orbitals,
        snakemake.output.azaphenalene_excitations,
    ),
    (
        "pentalene",
        mol_pentalene,
        snakemake.output.pentalene_orbitals,
        snakemake.output.pentalene_excitations,
    ),
    (
        "pentalene_ideal",
        mol_pentalene_ideal,
        snakemake.output.pentalene_ideal_orbitals,
        snakemake.output.pentalene_ideal_excitations,
    ),
)
variables = {}
for name, mol, path_orbitals, path_excitations in jobs:
    results, png_orbitals, png_excitations = compute_mol_qualitative(mol)

    # Write png files to disk
    with open(path_orbitals, "wb") as f:
        f.write(png_orbitals)
    with open(path_excitations, "wb") as f:
        f.write(png_excitations)

    # Collect all variables
    for key, value in results.items():
        variables[f"{name}_{key}"] = value
    variables[f"fig_{name}_orbitals"] = "../" + str(Path(path_orbitals).with_suffix(""))
    variables[f"fig_{name}_excitations"] = "../" + str(
        Path(path_excitations).with_suffix("")
    )

Take out reference data

In [ ]:
# snakemake.input.ref_azaphenalene
df = pd.read_csv(snakemake.input.azaphenalene_ref, index_col=0)
variables["azaphenalene_t1_s1_ref"] = f"{df.loc[1]['t1_s1_ref'] * EV_TO_KCAL:.2f}"

# snakemake.input.ref_pentalene
df = pd.read_csv(snakemake.input.pentalene_ref, index_col=0)
variables["pentalene_t1_s1_ref"] = f"{df.loc['IV_1']['t1_s1_ref'] * EV_TO_KCAL:.2f}"

Write everything YAML for processing by article

In [ ]:
with open(snakemake.output.variables, "w") as f:
    yaml.dump(variables, f)